In [ ]:
import agama
import numpy as np
import matplotlib.pyplot as plt

# Examples of the ActionFinder including finding orbit paramters

This notebook has been written primarily to demonstrate the ability to derive orbit parameters using the Staeckel fudge, added by Paul McMillan 04/2022, following the approach of [Mackereth & Bovy (2018)](https://ui.adsabs.harvard.edu/abs/2018PASP..130k4501M/abstract)

#### Set units mass=Msun, length = 1 kpc, velocity = km/s

In [ ]:
agama.setUnits(mass=1, length=1, velocity=1)

#### Define gravitational potential (e.g., the one from McMillan 2017), and initiate the object that determines actions in this potential

In [ ]:
pot= agama.Potential('data/McMillan17.ini')
af_fudge = agama.ActionFinder(pot)

## Example 1: The orbit of the Sun

In [ ]:
Sun_v = np.array([8.2,0,0.014,11.1,-245,7.25])

In [ ]:
J,theta,orbpar,freq = af_fudge(Sun_v,angles=True,orbitparameters=True)
print('J_R=%.2f, J_z=%.2f, J_phi=%.1f km/s/kpc' % (J[0],J[1],J[2]))
print('theta_R, theta_z, theta_phi = %.3f, %.3f, %.3f' % (theta[0], theta[1], theta[2]))
print('Rmin, Rmax, Zmax = %.3f, %.3f, %.3f kpc' % (orbpar[0], orbpar[1], orbpar[2]))
print('omega_R, omega_Z, omega_phi = %.3f, %.3f, %.3f /Myr' % (freq[0], freq[1], freq[2]))


## Example 2: An orbit with escape velocity (10$\times$ velocity of the Sun)

In [ ]:
Escape_v = np.array([8.2,0,0.014,11.1,-2450,7.25])
J,theta,orbpar,freq = af_fudge(Escape_v,angles=True,orbitparameters=True)
print('J_R=%.2f, J_z=%.2f, J_phi=%.1f km/s/kpc' % (J[0],J[1],J[2]))
print('theta_R, theta_z, theta_phi = %.3f, %.3f, %.3f' % (theta[0], theta[1], theta[2]))
print('Rmin, Rmax, Zmax = %.3f, %.3f, %.3f kpc' % (orbpar[0], orbpar[1], orbpar[2]))
print('omega_R, omega_Z, omega_phi = %.3f, %.3f, %.3f /Myr' % (freq[0], freq[1], freq[2]))


Note that actions etc are undefined for this orbit.

One of the values given here should actually not be undefined, and it is a shortcoming of the approach that it is given as nan. Which one?

## Example 1, continued

In [ ]:
times,points = agama.orbit(ic=Sun_v,potential=pot,time=1,trajsize=1001)
J,theta,orbpar,freq = af_fudge(Sun_v,angles=True,orbitparameters=True)

### Compare integrated orbit in R,z with Rmin, Rmax, Zmax computed

In [ ]:
plt.plot((points[:,0]**2+points[:,1]**2)**0.5,points[:,2])
plt.plot([orbpar[0],orbpar[0],orbpar[1],orbpar[1]],[orbpar[2],-orbpar[2],orbpar[2],-orbpar[2]], 'kx')
plt.xlabel('R')
plt.ylabel('z')
plt.show()

(N.B., there is an important bad choice I have made to show the above plot, which does make it easier to read. What is it?)

In [ ]:
# integrate for longer so we can be sure we cover the whole orbit
times,points = agama.orbit(ic=Sun_v,potential=pot,time=100,trajsize=10001)
print( 'Zmax: %.5f vs %.5f' % (np.max(points[:,2]), orbpar[2]))

R = (points[:,0]**2+points[:,1]**2)**0.5
print( 'Rmin: %.5f vs %.5f' % (np.min(R), orbpar[0]))
print( 'Rmax: %.5f vs %.5f' % (np.max(R), orbpar[1]))

So these are pretty good approximations. There will be orbits where it does not work so well.

# Now we look at actions, and try to develop some intuative understanding

### The easiest way to do so is to look at a 'surface of section'

Choose a line thats passes through the orbit in the R-z plane, and when the orbit passes it plot the position along the line and the velocity along the line,

e.g., the line z=0, so we're plotting R, vR


In [ ]:
def polyArea(X,Y):
    '''Function that returns the area of a polygon from the positions of its verticies'''
    return 0.5*np.abs(np.sum(X[1:]*Y[:-1] - Y[1:]*X[:-1]) + X[0]*Y[-1] - Y[0]*X[-1])

# quick and dirty, take points where distance z=0 is very small
mask = np.abs(points[:,2] ) < 0.001
vR = (points[:,0]*points[:,3] + points[:,1]*points[:,4])/R
RSOS, vRSOS = R[mask],vR[mask]
plt.plot(RSOS,vRSOS,'.')
plt.xlabel('R')
plt.ylabel('vR')

sorter = np.arctan2(vRSOS, RSOS - np.mean(RSOS))
argsort = np.argsort(sorter)
RSOS, vRSOS = RSOS[argsort], vRSOS[argsort]
print('Area >= %.2f' % polyArea(RSOS,vRSOS) )
print('Jr*2pi = %.2f' % (J[0] *2*np.pi))
plt.show()

The area inside the curve is $2\pi J_r$

Similarly, if we do the same at R=R_divide (where R_divide is in the range where the line will cut through the top & bottom of the orbit)

In [ ]:
R_divide = 8.6
# quick and dirty again
mask = np.abs(R-R_divide) < 0.005
ZSOS, vZSOS = (points[:,2])[mask], (points[:,5])[mask]
plt.plot(ZSOS,vZSOS,'.')
plt.xlabel('Z')
plt.ylabel('vZ')

sorter = np.arctan2(vZSOS, ZSOS)
argsort = np.argsort(sorter)
ZSOS, vZSOS = ZSOS[argsort], vZSOS[argsort]
print('Area >= %.2f' % polyArea(ZSOS,vZSOS) )
print('Jz*2pi = %.2f' % (J[1] *2*np.pi))
plt.show()

Note that we could have picked z= z_divide for any z_divide which intersects the right and left sides of the orbit, and an r=R_divide anywhere that intersects the top and bottom of the orbit.

Try it for yourself!

Indeed any sloped (or even curved) line in the R-z plane (as long as again it intersected right & left or top & bottom) would work. In that case we would need to be careful that the velocity we plotted was the velocity along that line (which would be function of, at minimum, vZ and vR)

## As velocities get further from that of a circular orbit, the orbit paths can get complicated, e.g., below one has to be careful to intersect top and bottom, and not leave region the orbit covers

In [ ]:
v_alt = np.array([8.2,0,0.014,2.1,-220,120.25])
times,points = agama.orbit(ic=v_alt,potential=pot,time=3,trajsize=1001)
J,theta,orbpar,freq = af_fudge(v_alt,angles=True,orbitparameters=True)

plt.plot((points[:,0]**2+points[:,1]**2)**0.5,points[:,2])
plt.plot([orbpar[0],orbpar[0],orbpar[1],orbpar[1]],[orbpar[2],-orbpar[2],orbpar[2],-orbpar[2]], 'kx')
plt.xlabel('R')
plt.ylabel('z')
#plt.gca().set_aspect(1)
plt.show()


# And life can get more complicated still. For example, what's going on with this orbit?

In [ ]:
v_alt = np.array([8.2,0,0.014,50.1,-150,77.5])
J,theta,orbpar,freq = af_fudge(v_alt,angles=True,orbitparameters=True)
freq
times,points = agama.orbit(ic=v_alt,potential=pot,time=3,trajsize=1001)
plt.plot((points[:,0]**2+points[:,1]**2)**0.5,points[:,2])
plt.plot([orbpar[0],orbpar[0],orbpar[1],orbpar[1]],[orbpar[2],-orbpar[2],orbpar[2],-orbpar[2]], 'kx')
plt.xlabel('R')
plt.ylabel('z')
#plt.gca().set_aspect(1)

plt.show()

### If I plot it's surface of section...

In [ ]:
mask = np.abs(points[:,2] ) < 0.1
R = (points[:,0]**2+points[:,1]**2)**0.5
vR = (points[:,0]*points[:,3] + points[:,1]*points[:,4])/R
RSOS, vRSOS = R[mask],vR[mask]
plt.plot(RSOS,vRSOS,'.')
plt.xlabel('R')
plt.ylabel('vR')
plt.show()

Hmm. Maybe I should look at its orbital frequencies...